# Scikit-learnの学習と推論

scikit-learn用のSageMakerのコンテナは用意されていませんが、Chainer用のコンテナが非常にシンプルな作りになっているので、エントリーポイントを工夫してscikit-learnを扱えるコンテナにします。

## データのダウンロード

SVMの基本用途として2値分類を行います。ここではキノコのデータから、そのキノコが食べられるのか、食べられないのかを判定します。詳細は[こちら](https://archive.ics.uci.edu/ml/datasets/mushroom)をご覧ください。データはlibsvmのサイトのものを利用します。

データはlibsvm形式なので、scikit-learnが扱いやすいようにnumpyのファイル形式(.npz)にします。


In [ ]:
import os
import urllib.request
from sklearn.datasets import load_svmlight_file


os.makedirs("data/", exist_ok=True)
# downlaod data -> X:feature, y: label
data_url ="https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/mushrooms"
urllib.request.urlretrieve(data_url, "./data/mushroom")
data = load_svmlight_file("./data/mushroom")
X, y = data[0].todense(),data[1]

# split data into validation data and training data
import numpy as np
np.random.seed(seed=1)
n_alldata = X.shape[0]
n_val = int(n_alldata * 0.1)
n_test = int(n_alldata * 0.1)

# Draw validation data from alldata
val_index = np.random.choice(n_alldata, n_val)
# Draw test data after removing val data
rest = np.setdiff1d(range(n_alldata), val_index)
test_index = np.random.choice(rest, n_test)
# Training_data is the rest of data
train_index = np.setdiff1d(rest, test_index)

X_train = X[train_index]
y_train = y[train_index]
X_val = X[val_index]
y_val = y[val_index]
X_test = X[test_index]
y_test = y[test_index]

os.makedirs("data/train", exist_ok=True)
np.savez('./data/train/train.npz', feature=X_train, label=y_train)
os.makedirs("data/val", exist_ok=True)
np.savez('./data/val/val.npz', feature=X_val, label=y_val)

## データのアップロード

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()

train_input = sagemaker_session.upload_data(
    path=os.path.join('./data', 'train'),
    key_prefix='sklearn/mushroom/train')
val_input = sagemaker_session.upload_data(
    path=os.path.join('./data', 'val'),
    key_prefix='sklearn/mushroom/val')
print("training data is uploaded to {}".format(train_input))
print("validation data is uploaded to {}".format(val_input))


## 学習

Chainerのコンテナを利用します。その理由として：
- Chainerのコンテナを利用する場合は、学習用のコードを``if __name__=='__main__':``に書けばよいだけなので非常に制約が緩い。その関数の中身がchainerである必要もないのでscikit-learnのコードを書ける。
- scikit-learnのアルゴリズムは分散学習に対応していないので、分散学習のコードを必要としない。
- 唯一、推論用に用意された``default_predict_fn``がchainerのモデルを読み込めるようにchainerに特化した形で定義されている。この関数は、エントリーポイント内の``predict_fn``でoverrideされるので、chainerのモデルではなくscikit-learnのモデルを読めるように定義する。

コンテナ内でscikit-learnを使うために`pip install scikit-learn`をコンテナ内で実行する必要がありますが、`requirements.txt`に
```
scikit-learn
```
と1行書いて、エントリーポイントと同じ``source_dir``に置くと、コンテナ起動時にpipで自動インストールされます。

(補足: 2018/10/27時点)
- `SVM`と`RandomForest`を実装しています。hyperparameterで`{'algorithm': 'SVM'}`や`{'algorithm': 'RandomForest'}`のように指定してください。
- SVMで99.7%、RandomForestで100%(!)の精度が出ることがあります。


In [ ]:
from sagemaker import get_execution_role
from sagemaker.chainer.estimator import Chainer
role = get_execution_role()

instance_type = 'ml.m4.xlarge'
sklearn_estimator = Chainer(entry_point='mushroom.py',
                            source_dir='source_dir',
                            role=role,
                            train_instance_count=1, 
                            train_instance_type=instance_type,
                            hyperparameters={'algorithm': 'RandomForest'})

sklearn_estimator.fit({'train': train_input, 'test': val_input})

## 推論

`deploy`という関数を呼び出してエンドポイントを作成します。エントリーポイント内の`model_fn`と`predict_fn`を見るとわかりますが、エンドポイント側の処理をまとめると以下のようになります。単純にS3にアップロードされているモデル(pkl)を読み出して、scikit-learnのpredictを実行しているだけです。
```python
with  open(os.path.join(model_dir, 'model.pkl'), 'rb') as pickle_file:
    model = pickle.load(pickle_file)
return  model.predict(data)    
```

In [ ]:
instance_type = 'ml.m4.xlarge'
predictor = sklearn_estimator.deploy(initial_instance_count=1, instance_type=instance_type)

あらかじめ用意しておいたテストデータ`X_test`をエンドポイントに送信して予測結果を得ます。1が毒キノコ、2が食べられるキノコを表しています。

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = predictor.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(accuracy))
print(y_pred)
